In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Configuration
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
rootpath = os.path.dirname(os.getcwd())
sys.path.append(rootpath)

from cci import MyIO, MyFunc, MyPath
myio = MyIO()
myfunc = MyFunc()
mypath = MyPath()

import numpy as np
import itertools
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, SimpleRNN
from tensorflow.keras.optimizers import Adam, schedules
from tensorflow.keras.callbacks import ModelCheckpoint

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/fsspec/__init__.py:47: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for spec in entry_points.get("fsspec.specs", []):


In [2]:
## Parameters
TOPN = 1000
RANDOM_STATE = 42

## Filenames
fname_data_norm = f'data_w-{TOPN}_norm.pk'
fname_corr_vars = 'correlated_variables.json'

fdir_reg_model = os.path.sep.join((mypath.fdir_model, 'regression'))

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
df_norm = myio.load(fname_object=fname_data_norm, _type='data')

  | fdir : /data/blank54/workspace/project/cci/data
  | fname: data_w-1000_norm.pk


/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
corr_vars = myio.load_json(fname_object=fname_corr_vars, _type='data')
print(f'  | # of correlated variables: {len(corr_vars)}')

  | fdir : /data/blank54/workspace/project/cci/data
  | fname: correlated_variables.json
  | # of correlated variables: 342


/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
x_df = df_norm[corr_vars]
y_df = myio.load_cci(start='200502', end='201912')['cci']

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, shuffle=False, random_state=RANDOM_STATE)

print('X variables: {}'.format(x_df.shape))

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


X variables: (179, 342)


In [6]:
def dnn_model_large(INPUT_SIZE):
    model = keras.Sequential()
    model.add(Dense(units=256, activation='relu', input_shape=(INPUT_SIZE,)))
    model.add(Dropout(0.3))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def dnn_model_small(INPUT_SIZE):
    model = keras.Sequential()
    model.add(Dense(units=32, activation='relu', input_shape=(INPUT_SIZE,)))
    model.add(Dropout(0.3))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=8, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=4, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def cnn_model_large(INPUT_SIZE):
    model = keras.Sequential()
    model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(INPUT_SIZE,1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def cnn_model_small(INPUT_SIZE):
    model = keras.Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(INPUT_SIZE,1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def rnn_model_large(INPUT_SIZE):
    model = keras.Sequential()
    model.add(SimpleRNN(128, input_shape=(INPUT_SIZE, 1)))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def rnn_model_small(INPUT_SIZE):
    model = keras.Sequential()
    model.add(SimpleRNN(64, input_shape=(INPUT_SIZE, 1)))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='linear'))
    
    return model

def train(model, x_train, y_train, BATCH_SIZE, LEARNING_RATE, NUM_EPOCHS):
    fname_model = f'{MODEL_TYPE}_model_{MODEL_SIZE}_B-{BATCH_SIZE}_L-{LEARNING_RATE}_E-{NUM_EPOCHS}.h5'
    fpath_model = os.path.sep.join((fdir_reg_model, fname_model))
    
    model_checkpoint = ModelCheckpoint(fpath_model, monitor='val_loss', mode='min', save_best_only=True)
    lr_schedule = schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, 
                                             decay_steps=100,
                                             decay_rate=0.9)
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mse')
    history = model.fit(x_train, y_train,
                        batch_size=BATCH_SIZE,
                        epochs=NUM_EPOCHS,
                        validation_split=0.3,
                        verbose=0,
                        callbacks=[model_checkpoint])
    
    return history

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

def plot_history(history, target):
    plt.figure(figsize=(16,7))

    plt.plot(range(len(history.history[target])), history.history[target], label=f'Train {target}')
    plt.plot(range(len(history.history[target])), history.history[f'val_{target}'], label=f'Valid {target}')
    
    plt.ylabel(target)
        
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    
def train_model(MODEL_TYPE, MODEL_SIZE):
    global BATCH_SIZE_LIST, LEARNING_RATE_LIST, NUM_EPOCHS_LIST, INPUT_SIZE

    
    if MODEL_TYPE == 'dnn' and MODEL_SIZE == 'large':
        model = dnn_model_large(INPUT_SIZE)
    elif MODEL_TYPE == 'dnn' and MODEL_SIZE == 'small':
        model = dnn_model_small(INPUT_SIZE)
    elif MODEL_TYPE == 'cnn' and MODEL_SIZE == 'large':
        model = cnn_model_large(INPUT_SIZE)
    elif MODEL_TYPE == 'cnn' and MODEL_SIZE == 'small':
        model = cnn_model_small(INPUT_SIZE)
    elif MODEL_TYPE == 'rnn' and MODEL_SIZE == 'large':
        model = rnn_model_large(INPUT_SIZE)
    elif MODEL_TYPE == 'rnn' and MODEL_SIZE == 'small':
        model = rnn_model_small(INPUT_SIZE)
    else:
        return None
    
    mape_list = []
    for BATCH_SIZE, LEARNING_RATE, NUM_EPOCHS in itertools.product(BATCH_SIZE_LIST, LEARNING_RATE_LIST, NUM_EPOCHS_LIST):
        history = train(model, x_train, y_train, BATCH_SIZE, LEARNING_RATE, NUM_EPOCHS)

        y_pred = model.predict(x_test)
        mape = MAPE(y_test.values, y_pred)
        mape_list.append(mape)

        fname_history = f'dnn_history_small_B-{BATCH_SIZE}_L-{LEARNING_RATE}_E-{NUM_EPOCHS}.pk'
        myio.save(_object=history, fdir_object=fdir_reg_model, fname_object=fname_history)

        print('============================================================')
        print('Model training')
        print(f'  | Batch size: {BATCH_SIZE}')
        print(f'  | Learning rate: {LEARNING_RATE}')
        print(f'  | Num epochs: {NUM_EPOCHS}')
        print(f'  | MAPE: {mape:.03f}%')
        print('============================================================')
        
    return mape_list

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
## Model parameters
INPUT_SIZE = x_train.shape[1]

BATCH_SIZE_LIST = [8, 16, 32]
LEARNING_RATE_LIST = [1e-3, 1e-4, 1e-5]
NUM_EPOCHS_LIST = [100, 1000, 10000]

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
MODEL_TYPE = 'rnn'
MODEL_SIZE = 'small'

mape_cnn_small = train_model(MODEL_TYPE, MODEL_SIZE)

/data/blank54/anaconda3/envs/cci/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


INFO:tensorflow:Assets written to: ram://35d2f1a7-333e-4847-9949-e69e0ecfa1d9/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-8_L-0.001_E-100.pk
Model training
  | Batch size: 8
  | Learning rate: 0.001
  | Num epochs: 100
  | MAPE: 29.158%
INFO:tensorflow:Assets written to: ram://08265f42-ec59-4dfe-a9cd-4dc2210f8f06/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-8_L-0.001_E-1000.pk
Model training
  | Batch size: 8
  | Learning rate: 0.001
  | Num epochs: 1000
  | MAPE: 17.669%
INFO:tensorflow:Assets written to: ram://3e0d4550-1121-4ddb-ac37-b2ddae1b8a36/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-8_L-0.001_E-10000.pk
Model training
  | Batch size: 8
  | Learning rate: 0.001
  | Num epochs: 10000
  | MAPE: 18.680%
INFO:tensorflow:Assets written to: ram://d8b88a54-b29a-4f2c-9312-fd78fe157b8c/assets
  | fdir : /data/blank54

INFO:tensorflow:Assets written to: ram://ac1fdb59-a7e1-4577-a5fd-d01489948924/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-16_L-1e-05_E-1000.pk
Model training
  | Batch size: 16
  | Learning rate: 1e-05
  | Num epochs: 1000
  | MAPE: 18.600%
INFO:tensorflow:Assets written to: ram://17e76b15-0f13-4599-b543-f4477e912cb6/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-16_L-1e-05_E-10000.pk
Model training
  | Batch size: 16
  | Learning rate: 1e-05
  | Num epochs: 10000
  | MAPE: 18.557%
INFO:tensorflow:Assets written to: ram://0306c066-cbcb-4c8d-90e3-97975adfe230/assets
  | fdir : /data/blank54/workspace/project/cci/model/regression
  | fname: dnn_history_small_B-32_L-0.001_E-100.pk
Model training
  | Batch size: 32
  | Learning rate: 0.001
  | Num epochs: 100
  | MAPE: 19.965%
INFO:tensorflow:Assets written to: ram://e8c23d4b-1236-4f36-9975-e723ac1049d7/assets
  | fdir : /data/b